# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [1]:
def retrieve_titles(soup):
    imgs = soup.find_all('img', {'class': 'thumbnail'})
    return [t.attrs['alt'] for t in imgs]


## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [17]:
import re
def retrieve_ratings(soup):
    regex = re.compile("star-rating (.*)")
    ratings = []
    replace_dict = {'One' : 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five': 5}
    for rating in soup.findAll('p', {"class" : regex}):
        ratings.append(replace_dict[rating.attrs['class'][-1]])
    
    return ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [3]:
def retrieve_prices(soup):
    p = soup.findAll('p', {'class': 'price_color'})
    return [float(x.text[1:]) for x in p]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [4]:
def retrieve_availabilities(soup):
    a = soup.findAll('p', {'class': 'availability'})
    return [x.text.strip() for x in a]

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

r = requests.get('http://books.toscrape.com/catalogue/page-1.html')

                     

In [30]:
soup = BeautifulSoup(r.content, 'html.parser')
df = pd.DataFrame()
titles = retrieve_titles(soup)
ratings = retrieve_ratings(soup)
prices = retrieve_prices(soup)
avail = retrieve_availabilities(soup)
df_temp = pd.DataFrame([titles,ratings,prices,avail]).T
df_temp.columns = ['Title', 'Star Rating', 'Price in £', 'Stock Status']
df = df.append(df_temp)
df

,Title,Star Rating,Price in £,Stock Status
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


In [53]:
#Your code here

df = pd.DataFrame()
for i in range(1, 51):
    url = 'http://books.toscrape.com/catalogue/page-{}.html'.format(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser') 

    titles = retrieve_titles(soup)
    ratings = retrieve_ratings(soup)
    prices = retrieve_prices(soup)
    avail = retrieve_availabilities(soup)
                         
    df_temp = pd.DataFrame([titles,ratings,prices,avail]).T
    df_temp.columns = ['Title', 'Star Rating', 'Price in £', 'Stock Status']
    df = df.append(df_temp, ignore_index=True)
                         
df


,Title,Star Rating,Price in £,Stock Status
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [52]:
#Your code here
df2 = pd.DataFrame()
url = 'http://books.toscrape.com/catalogue/'
path = 'page-1.html'
for i in range(2, 54):
    response = requests.get(url + path)
    soup = BeautifulSoup(response.content, 'html.parser') 
    titles = retrieve_titles(soup)
    ratings = retrieve_ratings(soup)
    prices = retrieve_prices(soup)
    avail = retrieve_availabilities(soup)
                         
    df_temp = pd.DataFrame([titles,ratings,prices,avail]).T
    df_temp.columns = ['Title', 'Star Rating', 'Price in £', 'Stock Status']
    df2 = df2.append(df_temp, ignore_index=True)
    
    #set next url
    try:
        path = soup.find('li', {'class' : 'next'}).a.attrs['href']
    except:
        print('No pages left')
        break;
df2

No pages left


,Title,Star Rating,Price in £,Stock Status
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock


In [54]:
df == df2

,Title,Star Rating,Price in £,Stock Status
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
...,...,...,...,...
995,True,True,True,True
996,True,True,True,True
997,True,True,True,True
998,True,True,True,True


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!